In [22]:
from preprocess import drop_useless
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression


In [24]:
data_test=pd.read_csv('../../data/test_airbnb_berlin.xls')
data_train=pd.read_csv('../../data/train_airbnb_berlin.xls')

In [25]:
#Data visualisation

# computing number of rows
rows = len(data_train.axes[0])
print('number row :',rows)
# computing number of columns
cols = len(data_train.axes[1])
print('number columns :',cols)

number row : 15692
number columns : 39


In [26]:
data_train=drop_useless(data_train)

__Gestion des features à taux de "missing" extrême__

Lorsque l'on regarde les features où il manque moins de 30 données, on se rend compte que l'absence de ces données est MCAR. On se permet donc de supprimer les lignes où ces données sont manquantes. 
Il y a une exception sur la feature 'Is Superhost': si pour ces host il y a une note générale au dessus de 80, on les catégorises en Superhost, sinon, non. 

In [27]:
#Suppression des features à taux de missing trop élevé 
data_train=data_train.drop(['Square Feet','Host Response Time','Host Response Rate'], axis=1)

In [28]:
#Listwise deletion pour les features à taux de missing faible (borne max des rows concernés : 60 /15k = 0.4%)
for j in ['Price','Bathrooms','Bedrooms','Beds']:
    for i in data_train[data_train[j].isna()== True].index:
        data_train=data_train.drop(i, axis=0)

In [29]:
#plus ou moins valide pour 20 personnes
# Gestion des 'Is Superhost' : si le host est noté, et ce au dessus de 80, il est supposé superhost, sinon il ne l'est pas
for i in data_train[data_train['Is Superhost'].isna()== True].index:
    if data_train.loc[i,'Overall Rating']>=80:
        data_train.loc[i,'Is Superhost']='t'
    else:
        data_train.loc[i,'Is Superhost']='f'

On cherche a réaliser une regression multivariate 

On va prédire les features "Rating" de deux façons : soit colonne par colonne, soit toutes les colonnes à la fois.

In [30]:
data_train.columns.values

array(['Is Superhost', 'Neighborhood Group', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accomodates', 'Bathrooms',
       'Bedrooms', 'Beds', 'Guests Included', 'Min Nights', 'Reviews',
       'Overall Rating', 'Accuracy Rating', 'Cleanliness Rating',
       'Checkin Rating', 'Communication Rating', 'Location Rating',
       'Value Rating', 'Instant Bookable', 'Business Travel Ready',
       'Price'], dtype=object)

In [31]:
without_ratings=data_train.drop(['Overall Rating', 'Accuracy Rating', 'Cleanliness Rating',
       'Checkin Rating', 'Communication Rating', 'Location Rating','Value Rating'], axis=1)

data_train_clean_ratings_included=data_train

for j in ['Overall Rating', 'Accuracy Rating', 'Cleanliness Rating','Checkin Rating', 'Communication Rating', 'Location Rating','Value Rating']:
       for i in data_train_clean_ratings_included[data_train_clean_ratings_included[j].isna()== True].index:
              data_train_clean_ratings_included=data_train_clean_ratings_included.drop(i, axis=0)


In [32]:
X=data_train_clean_ratings_included.drop(['Overall Rating', 'Accuracy Rating', 'Cleanliness Rating','Checkin Rating', 'Communication Rating', 'Location Rating','Value Rating'],axis=1)
y=data_train_clean_ratings_included['Overall Rating']

print('X:',X)
print('y',y)

X:       Is Superhost        Neighborhood Group  Latitude  Longitude  \
0                f                    Pankow  52.54652   13.41792   
1                f                    Pankow  52.56512   13.42214   
2                f                    Pankow  52.54741   13.42521   
3                f  Friedrichshain-Kreuzberg  52.50958   13.45144   
5                t   Tempelhof - SchÃ¶neberg  52.44826   13.40608   
...            ...                       ...       ...        ...   
15685            f                     Mitte  52.52759   13.33738   
15686            f                     Mitte  52.52638   13.39698   
15687            f  Friedrichshain-Kreuzberg  52.49016   13.39241   
15688            f                 NeukÃ¶lln  52.47586   13.45362   
15690            f   Tempelhof - SchÃ¶neberg  52.47179   13.33103   

      Property Type        Room Type Accomodates Bathrooms Bedrooms Beds  \
0         Apartment     Private room           2       1.0      1.0  1.0   
1         Apartm

In [21]:
it_imp = IterativeImputer(sample_posterior=True)
X_it_imp = it_imp.fit_transform(np.append(X, np.reshape(y, (-1, 1)), axis=1))

ValueError: Data must be 1-dimensional